In [ ]:
# 라이브러리 import 및 logging 설정
import requests
import time
import logging

import data_utils

data_utils.setup_logging()
logging.info("필수 라이브러리 및 data_utils 모듈 임포트, 로깅 설정 완료")

In [ ]:
# 직종 구분 선택 -> 사이트마다 다르게 설정할 것
job_category = "total" # "total" # "backend" # "frontend"

if job_category == "total":
    job_category_url = "?"
elif job_category == "backend":
    job_category_url = "?jobCategory=1&"
elif job_category == "frontend":
    job_category_url = "?jobCategory=2&"
else:
    job_category_url = "?"

In [ ]:
# 스크래핑 설정
headers = data_utils.DEFAULT_HEADERS
site_name = 'jumpit'

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.info("스크래핑 설정 완료.")

In [ ]:
def scrape_jumpit_positions():
    """Jumpit 채용 공고를 스크래핑하여 데이터 리스트를 반환합니다."""
    all_positions_data = []
    page_no = 1

    logging.info("Jumpit 채용 공고 스크래핑 시작")

    while True:
        url = f"https://jumpit-api.saramin.co.kr/api/positions{job_category_url}sort=rsp_rate&highlight=false&page={page_no}"
        logging.info(f"페이지 {page_no}의 JSON 데이터 요청: {url}")

        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()

            data = response.json()

            positions_list = data['result']['positions']

            if not positions_list:
                logging.info("더 이상 채용 공고가 없습니다. 스크래핑 종료.")
                break

            logging.info(f"페이지 {page_no}에서 {len(positions_list)}개의 채용 공고 발견")

            for position in positions_list:
                company_name = position['companyName']
                position_title = position['title']
                tech_stacks = position['techStacks']
                skill = ', '.join(tech_stacks) if tech_stacks else "N/A"

                # --- 여기에 필터링 함수 적용 ---
                filtered_skill = data_utils.filter_skill_data(skill)
                # --- 필터링 완료 ---

                position_data = {
                    'company': company_name,
                    'position': position_title,
                    'skill': filtered_skill,
                }
                all_positions_data.append(position_data)

            page_no += 1
            time.sleep(1)

        except requests.exceptions.RequestException as e:
            logging.error(f"페이지 {page_no} 요청 실패: {e}")
            break
        except KeyError as e:
            logging.error(f"페이지 {page_no} JSON 파싱 에러 (키 에러): {e}")
            break
        except Exception as e:
            logging.error(f"예상치 못한 에러 발생: {e}")
            break

    return all_positions_data

In [ ]:
# 두 번째 셀을 실행하여 스크래핑된 데이터를 가져옵니다.
scraped_data = scrape_jumpit_positions()

In [ ]:
# data_utils 모듈의 save_data_to_csv 함수를 사용하여 데이터를 저장합니다.
data_folder = 'data'
filename = f'data_{site_name}_{job_category}.csv'

saved_filepath = data_utils.save_data_to_csv(scraped_data, filename, folder=data_folder)

In [ ]:
# 저장된 CSV 파일을 읽어와 DataFrame으로 출력 (확인용)
data_utils.load_data_from_csv(saved_filepath)